In [31]:
import utils
import os
import yaml
import argparse
import math
import warnings
from torch.utils.data import DataLoader
from datasets import *
from model_builder import *
import json

In [32]:
CONFIGS = utils.load_config_file('configs/configs.yaml')
LOOP = 1
EXP = 'exp1'

In [34]:
for i in range(1,10):
    text = str(i) + ('_' + str(i))*(LOOP-1)
    CONFIGS['Dataset']['path'] = '/loop%s/%s' % (LOOP, text)
    CONFIGS['AudioProcessing']['duration'] = LOOP
    CONFIGS['RunningFolder']['run_path'] = os.path.join('pretrained_checkpoints',EXP,'loop%s/%s' % (LOOP, text))
    CONFIGS['Pairs']['Male']['eval_list'] =  os.path.join(CONFIGS['Dataset']['root_dir'],'loop%s/%s' % (LOOP, text),'male_eval.txt')
    CONFIGS['Pairs']['Female']['eval_list'] =  os.path.join(CONFIGS['Dataset']['root_dir'],'loop%s/%s' % (LOOP, text),'female_eval.txt')
    args = {'info_data':'meta_data/stage2/split_0.5_stage2.json',
        'stage':2,
        'eval':True,
        'set':'eval',
        'df_train':'data/loop1/0/background.csv',
        'dataset_name':'audio_mnist',
        'init_model':'pretrained_checkpoints/%s/loop%s/%s/model_best_acc.model' % (EXP, LOOP, text)
    }
    print(CONFIGS)
    break
    print('EVAL', text)
    eval_data(configs=CONFIGS, args=args)
    

    

{'Dataset': {'root_dir': 'data', 'noise_test_paths': ['../dataset/speech_commands_v0.01/_background_noise_'], 'rir_path': '../dataset/rir_data/RIRS_NOISES/simulated_rirs', 'musan_path': '../dataset/musan_data/musan', 'path': '/loop1/1'}, 'RunningFolder': {'run_path': 'pretrained_checkpoints/exp1/loop1/1', 'score_file': 'score-stage.txt', 'threshold_file': 'threshold-tuning.txt'}, 'Pairs': {'threshold_path': 'pretrained_checkpoints/exp1/loop1/0/thresholds.json', 'Female': {'eval_list': 'data/loop1/1/female_eval.txt'}, 'Male': {'eval_list': 'data/loop1/1/male_eval.txt'}}, 'AudioProcessing': {'sample_rate': 16000, 'duration': 1, 'add_sample': 240}, 'Parameters': {'lr': 0.0001, 'lr_decay': 0.97, 'num_workers': 0, 'max_epoch': 35, 'batch_size': 32, 'scheduler': 'StepLR', 'base_lr': 1e-08, 'max_lr': 0.0001, 'lr_scheduler_patience': 1, 'lr_scheduler_gamma': 0.05, 'C': 1024, 'm': 0.2, 's': 30, 'test_step': 1, 'device': 'cuda:0'}}


In [26]:

def eval_data(configs,args):
    audio_cfgs = configs['AudioProcessing']
    dataset_cfgs = configs['Dataset']
    param_cfgs = configs['Parameters']
    folder_cfgs = configs['RunningFolder']

    eval_info = {
        'female': configs['Pairs']['Female'],
        'male': configs['Pairs']['Male'],

    }
    name_set = args['set']


    with open(args['info_data'], 'r') as file_in:
        info = json.load(file_in)

    # torch.manual_seed(args.seed)

    if args['stage'] == 1:
        if args['gender'] == 'mix':
            classes = info['speakers']
        elif args['gender'] == 'female':
            classes = info['female_speakers']
        else:
            classes = info['male_speakers']
    else:
        classes = info['development']['female'] + info['development']['male']
    n_class = len(classes)

    # train_transform = build_transform(audio_config=audio_cfgs,
    #                                     mode='train',
    #                                     noise_path=dataset_cfgs,
    #                                     stage=args.stage)
    # train_dataset = GeneralDataset(root_dir=dataset_cfgs['root_dir'],
    #                                 path_to_df=args.df_train,
    #                                 classes=classes,
    #                                 sample_rate=audio_cfgs['sample_rate'],
    #                                 dataset_name=args.dataset_name,
    #                                 stage=args.stage,
    #                                 transform=train_transform,
    #                                 gender=args.gender)

    # train_loader = DataLoader(train_dataset,
    #                             batch_size=param_cfgs['batch_size'],
    #                             num_workers=param_cfgs['num_workers'],
    #                             shuffle=True,
    #                             drop_last=True)
    # if args.stage == 1:
    #     valid_transform = build_transform(audio_config=audio_cfgs,
    #                                         mode='eval',
    #                                         noise_path=dataset_cfgs,
    #                                         stage=1)
    #     valid_dataset = GeneralDataset(root_dir=dataset_cfgs['root_dir'],
    #                                     path_to_df=args.df_valid,
    #                                     classes=classes,
    #                                     sample_rate=audio_cfgs['sample_rate'],
    #                                     dataset_name=args.dataset_name,
    #                                     stage=1,
    #                                     transform=valid_transform,
    #                                     gender=args.gender)
    #     valid_loader = DataLoader(valid_dataset,
    #                                 batch_size=param_cfgs['batch_size'],
    #                                 num_workers=param_cfgs['num_workers'],
    #                                 shuffle=False)

    model = ECAPAModel(configs=configs,
                        n_class=n_class)
    print('The number of classes', n_class)
    if args['eval']:
        print("Model %s loaded from previous state!" % args['init_model'])
        model.load_parameters(args['init_model'])
        if args['stage'] == 2:
            # score_file = open(folder_cfgs['run_path'] +
            #           '/' + folder_cfgs['threshold_file'], "a+")
            # print('Tune threshold', args.tune_threshold)
            # tuned_threshold = {'male':[], 'female':[]}
            sum_eer = 0
            sum_minDCF = 0
            # if args.tune_threshold:
            #     threshold_store = {}
            # else:
            # tuned_threshold = json.load(open(CONFIGS['Pairs']['threshold_path']))
            
            results = {}
            for gender in eval_info:
                # if not args.tune_threshold:
                #     print('Threshold', tuned_threshold[gender])
                eval_list = eval_info[gender]['%s_list'%(name_set)]
                EER, minDCF,thresholds = model.eval_eer(
                    eval_list=eval_list, eval_path=dataset_cfgs['root_dir'],
                    tuning=True,
                    thresholds=[1,1])
                
                results[gender] = {'eer': EER, 'minDCF': minDCF}
                
                sum_eer += EER
                sum_minDCF += minDCF
                sys.stderr.write("Gender %s, EER %2.2f%%, minDCF %.4f, threshold %s\n" % (gender, EER, minDCF,thresholds))
                sys.stderr.flush()
                # score_file.write("Gender %s, EER %2.2f%%, minDCF %.4f, threshold %s\n" % (gender, EER, minDCF,thresholds))
                # score_file.flush()
                
                # if args.tune_threshold:
                #     threshold_store[gender] = thresholds
            results['overall'] = {'eer': sum_eer/2, 'minDCF': sum_minDCF/2}
            json.dump(results, open(folder_cfgs['run_path'] + '/eval_results.json','w'))

            # if args.tune_threshold:
            #     json.dump(threshold_store, open(folder_cfgs['run_path'] + '/thresholds.json','w'))
        # else:
        #     res = model.eval_stage_1(
        #         valid_loader, classes=info['speakers'], path_to_result=args.path_to_result)
        quit()
    # print(args.init_model)
    if args.pretrained_model is not None:
        print("Model %s loaded from previous state!" % args.pretrained_model)
        model.load_parameters(args.pretrained_model)
    else:
        epoch = 1

    EERs = []
    if args.stage == 1:
        best_score = -math.inf
    else:
        # best_loss = math.inf
        best_acc =  -math.inf
        best_eer = math.inf
        best_DCF = math.inf
    if not os.path.exists(folder_cfgs['run_path']):
        os.mkdir(folder_cfgs['run_path'])

    score_file = open(folder_cfgs['run_path'] +
                        '/' + folder_cfgs['score_file'], "a+")
    score_file.write("Seed: %d\n" % args.seed)
    with open(folder_cfgs['run_path'] + '/config.yaml', 'w') as outfile:
        yaml.dump(CONFIGS, outfile, default_flow_style=False)
    epoch = 1
    while (1):
        # Training for one epoch
        loss, lr, acc = model.train_network(epoch=epoch, loader=train_loader)

        # Evaluation every [test_step] epochs
        if epoch % param_cfgs['test_step'] == 0:
            if args.stage == 2:
                if acc >= best_acc:
                    best_acc = acc
                    model.save_parameters(
                        folder_cfgs['run_path'] + "/model_best_acc.model")

                sys.stderr.write(time.strftime("%Y-%m-%d %H:%M:%S") +
                        "%d epoch, ACC %2.2f%%,BestACC %2.2f%%\n" % (epoch, acc, best_acc))
                sys.stderr.flush()

                score_file.write("%d epoch, LR %f, LOSS %f, ACC %2.2f%%, BestACC %2.2f%%\n" % (
                    epoch, lr, loss, acc, best_acc))
                score_file.flush()

                # sum_eer = 0
                # tuned_threshold = {'male':{}, 'female':{}}

                # print(time.strftime("%Y-%m-%d %H:%M:%S"),
                #       " %d epoch, ACC %2.2f%%, LOSS %f" % (epoch, acc, loss))
                # score_file.write("%d epoch, LR %f, LOSS %f, ACC %2.2f%%\n" % (
                #     epoch, lr, loss, acc))
                # score_file.flush()


                # for gender in eval_info:
                #     eval_list = eval_info[gender]['%s_list'%(name_set)]
                #     EER, minDCF, thresholds = model.eval_eer(eval_list=eval_list,
                #                                             eval_path=dataset_cfgs['root_dir'],
                #                                             tuning=True)
                #     sys.stderr.write(time.strftime("%Y-%m-%d %H:%M:%S") + 
                #                             " Gender %s, EER %2.2f%%, minDCF %.4f\n" % (gender, EER, minDCF))
                #     sys.stderr.flush()

                #     tuned_threshold[gender] = thresholds
                #     sum_eer += EER
                #     score_file.write("\tGender %s, EER %2.2f%%, minDCF %.4f, threshold %s\n" % (gender, EER, minDCF,thresholds))
                #     score_file.flush()


                # avg_eer = sum_eer / 2
                # if avg_eer < best_eer:
                #     best_eer = avg_eer
                #     model.save_parameters(
                #         folder_cfgs['run_path'] + "/model_best_eer.model")
                #     json.dump(tuned_threshold, open(folder_cfgs['run_path'] + '/thresholds.json','w'))
                
                # sys.stderr.write(time.strftime("%Y-%m-%d %H:%M:%S") + 
                #       " %d epoch, EER %2.2f%%, BEST_EER %2.2f%%" % (epoch, avg_eer, best_eer))
                # sys.stderr.flush()
                # score_file.write("%d epoch, EER %2.2f%%, BEST_EER %2.2f%%\n" % (epoch, avg_eer, best_eer))
                # score_file.flush()
                # if min_dcf < best_DCF:
                #     best_DCF = min_dcf
                #     model.save_parameters(
                #         folder_cfgs['run_path'] + "/model_best_dcf.model")

            else:
                _, val_acc = model.eval_acc(epoch=epoch, loader=valid_loader)
                if val_acc > best_score:
                    best_score = val_acc
                    model.save_parameters(
                        folder_cfgs['run_path'] + "/model_best.model")
                print(time.strftime("%Y-%m-%d %H:%M:%S"),
                        "%d epoch, ACC %2.2f%%,BestACC %2.2f%%" % (epoch, val_acc, best_score))
                score_file.write("%d epoch, LR %f, LOSS %f, ACC %2.2f%%, BestACC %2.2f%%\n" % (
                    epoch, lr, loss, acc, best_score))
                score_file.flush()

        if epoch >= param_cfgs['max_epoch']:
            quit()
        epoch += 1
